In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [2]:
weather = pd.read_csv("weather3_180703.csv")
weather['date'] = pd.to_datetime(weather["date"])
station = weather[weather['station_nbr'] == 17]

In [3]:
station['date'].count()

1035

In [4]:
# ls == 모든 값이 0인 것

ls = []
ls1 = []
for i in station.columns:
    if station[i].count() == 0:
        ls.append(i)
    if station[i].isna().sum() != 0:
        ls1.append(i)
ls, ls1

(['depart', 'sunrise', 'sunset', 'daytime'],
 ['depart', 'sunrise', 'sunset', 'resultdir', 'daytime'])

In [5]:
for i in station.columns:
    count_null = station[i].isna().sum()
    print(i, ":", count_null, "(",round((count_null / len(station) * 100),2),"%",")")

station_nbr : 0 ( 0.0 % )
date : 0 ( 0.0 % )
tmax : 0 ( 0.0 % )
tmin : 0 ( 0.0 % )
tavg : 0 ( 0.0 % )
depart : 1035 ( 100.0 % )
dewpoint : 0 ( 0.0 % )
wetbulb : 0 ( 0.0 % )
heat : 0 ( 0.0 % )
cool : 0 ( 0.0 % )
sunrise : 1035 ( 100.0 % )
sunset : 1035 ( 100.0 % )
codesum : 0 ( 0.0 % )
snowfall : 0 ( 0.0 % )
preciptotal : 0 ( 0.0 % )
stnpressure : 0 ( 0.0 % )
sealevel : 0 ( 0.0 % )
resultspeed : 0 ( 0.0 % )
resultdir : 4 ( 0.39 % )
avgspeed : 0 ( 0.0 % )
year : 0 ( 0.0 % )
month : 0 ( 0.0 % )
day : 0 ( 0.0 % )
relative_humility : 0 ( 0.0 % )
windchill : 0 ( 0.0 % )
daytime : 1035 ( 100.0 % )
week7 : 0 ( 0.0 % )
weekend : 0 ( 0.0 % )
code_change : 0 ( 0.0 % )
rainY : 0 ( 0.0 % )
otherY : 0 ( 0.0 % )
nothing : 0 ( 0.0 % )


In [7]:
train = pd.read_csv("train.csv")
train.date = pd.to_datetime(train.date)
train.tail()

,date,store_nbr,item_nbr,units
4617595,2014-10-31,45,107,0
4617596,2014-10-31,45,108,0
4617597,2014-10-31,45,109,0
4617598,2014-10-31,45,110,0
4617599,2014-10-31,45,111,0


In [8]:
key = pd.read_csv("key.csv")
station = station.merge(key)

In [9]:
station = station.merge(train)
station.tail()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr,units
630697,17,2014-10-31,59.0,34.0,46.5,NaN,32.0,41.0,18.0,0.0,...,NaN,4,0,2,1,0,0,34,107,0
630698,17,2014-10-31,59.0,34.0,46.5,NaN,32.0,41.0,18.0,0.0,...,NaN,4,0,2,1,0,0,34,108,0
630699,17,2014-10-31,59.0,34.0,46.5,NaN,32.0,41.0,18.0,0.0,...,NaN,4,0,2,1,0,0,34,109,0
630700,17,2014-10-31,59.0,34.0,46.5,NaN,32.0,41.0,18.0,0.0,...,NaN,4,0,2,1,0,0,34,110,0
630701,17,2014-10-31,59.0,34.0,46.5,NaN,32.0,41.0,18.0,0.0,...,NaN,4,0,2,1,0,0,34,111,0


In [10]:
station['log1p_units'] = np.log1p(station.units)

In [11]:
target1 = station['units']
target2 = station['log1p_units']

station.drop(columns=['units','log1p_units'],inplace=True)
station.tail()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,windchill,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr
630697,17,2014-10-31,59.0,34.0,46.5,NaN,32.0,41.0,18.0,0.0,...,41.963648,NaN,4,0,2,1,0,0,34,107
630698,17,2014-10-31,59.0,34.0,46.5,NaN,32.0,41.0,18.0,0.0,...,41.963648,NaN,4,0,2,1,0,0,34,108
630699,17,2014-10-31,59.0,34.0,46.5,NaN,32.0,41.0,18.0,0.0,...,41.963648,NaN,4,0,2,1,0,0,34,109
630700,17,2014-10-31,59.0,34.0,46.5,NaN,32.0,41.0,18.0,0.0,...,41.963648,NaN,4,0,2,1,0,0,34,110
630701,17,2014-10-31,59.0,34.0,46.5,NaN,32.0,41.0,18.0,0.0,...,41.963648,NaN,4,0,2,1,0,0,34,111


In [12]:
df1 = pd.concat([station,target1], axis=1)
df2 = pd.concat([station,target2], axis=1)

In [13]:
df2.to_csv('station17.csv', sep=',', index=False)

In [14]:
if df2.columns.tolist() and ls:
    print('공통 변수 있음')
    print(df2.columns.tolist() and ls)
else:
    print('공통 변수 없음')

공통 변수 있음
['depart', 'sunrise', 'sunset', 'daytime']


In [15]:
df2.columns

Index(['station_nbr', 'date', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'snowfall',
       'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed', 'year', 'month', 'day', 'relative_humility', 'windchill',
       'daytime', 'week7', 'weekend', 'code_change', 'rainY', 'otherY',
       'nothing', 'store_nbr', 'item_nbr', 'log1p_units'],
      dtype='object')

In [16]:
ls, ls1

(['depart', 'sunrise', 'sunset', 'daytime'],
 ['depart', 'sunrise', 'sunset', 'resultdir', 'daytime'])

### 1. 변수변환: df2 (log1p_units)

In [17]:
all_var = 'log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(depart) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(sunrise) + scale(sunset) + scale(snowfall) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + C(resultdir) + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + C(daytime) + 0'

In [18]:
model2 = sm.OLS.from_formula('log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(snowfall) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2)
result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.609
Model:                            OLS   Adj. R-squared:                  0.609
Method:                 Least Squares   F-statistic:                     6816.
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:36:56   Log-Likelihood:            -1.8280e+05
No. Observations:              630702   AIC:                         3.659e+05
Df Residuals:                  630557   BIC:                         3.675e+05
Df Model:                         144                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(year)[2012]           

### 2. 변수변환 : df2 (log1p_units) + 아웃라이어 제거

In [19]:
# 아웃라이어 제거
# Cook's distance > 2 인 값 제거
influence = result2.get_influence()
cooks_d2, pvals = influence.cooks_distance
fox_cr = 4 / (len(df2) - 2)
idx_outlier = np.where(cooks_d2 > fox_cr)[0]
len(idx_outlier)

18887

In [20]:
idx = list(set(range(len(df2))).difference(idx_outlier))
df2_1 = df2.iloc[idx, :].reset_index(drop=True)
df2_1

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr,log1p_units
0,17,2012-01-01,56.0,38.0,47.0,NaN,23.0,37.0,18.0,0.0,...,NaN,6,1,0,0,0,1,9,1,0.0
1,17,2012-01-01,56.0,38.0,47.0,NaN,23.0,37.0,18.0,0.0,...,NaN,6,1,0,0,0,1,9,2,0.0
2,17,2012-01-01,56.0,38.0,47.0,NaN,23.0,37.0,18.0,0.0,...,NaN,6,1,0,0,0,1,9,3,0.0
3,17,2012-01-01,56.0,38.0,47.0,NaN,23.0,37.0,18.0,0.0,...,NaN,6,1,0,0,0,1,9,4,0.0
4,17,2012-01-01,56.0,38.0,47.0,NaN,23.0,37.0,18.0,0.0,...,NaN,6,1,0,0,0,1,9,6,0.0
5,17,2012-01-01,56.0,38.0,47.0,NaN,23.0,37.0,18.0,0.0,...,NaN,6,1,0,0,0,1,9,7,0.0
6,17,2012-01-01,56.0,38.0,47.0,NaN,23.0,37.0,18.0,0.0,...,NaN,6,1,0,0,0,1,9,8,0.0
7,17,2012-01-01,56.0,38.0,47.0,NaN,23.0,37.0,18.0,0.0,...,NaN,6,1,0,0,0,1,9,11,0.0
8,17,2012-01-01,56.0,38.0,47.0,NaN,23.0,37.0,18.0,0.0,...,NaN,6,1,0,0,0,1,9,12,0.0
9,17,2012-01-01,56.0,38.0,47.0,NaN,23.0,37.0,18.0,0.0,...,NaN,6,1,0,0,0,1,9,13,0.0


In [21]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(snowfall) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                 1.803e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:38:29   Log-Likelihood:             1.1609e+06
No. Observations:              611815   AIC:                        -2.322e+06
Df Residuals:                  611670   BIC:                        -2.320e+06
Df Model:                         144                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(year)[2012]           

In [22]:
ls1

['depart', 'sunrise', 'sunset', 'resultdir', 'daytime']

### 3. VIF 확인

In [23]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# all_cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
#         'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
#         'windchill', 'depart', 'sunrise', 'sunset', 'snowfall' ]
cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
        'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
        'windchill', 'snowfall' ]

y = df2_1.loc[:,cols]
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(y.values, i) for i in range(y.shape[1])]
vif["features"] = y.columns
vif = vif.sort_values("VIF Factor", ascending=False).reset_index(drop=True)
vif

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,VIF Factor,features
0,inf,tmax
1,inf,tmin
2,inf,tavg
3,1.307382e+06,sealevel
4,1.286596e+06,stnpressure
5,4.762758e+03,windchill
6,2.965890e+03,dewpoint
7,2.336441e+03,wetbulb
8,8.036914e+02,relative_humility
9,2.117969e+02,heat


### 4. VIF 근거, 빼고 다시 Model 돌림.

In [24]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(cool)\
                            + scale(snowfall) + scale(preciptotal) + scale(resultspeed)  \
                            + scale(avgspeed) + C(year) + C(month) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                 1.923e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:39:11   Log-Likelihood:             1.1609e+06
No. Observations:              611815   AIC:                        -2.322e+06
Df Residuals:                  611679   BIC:                        -2.320e+06
Df Model:                         135                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(year)[2012]          0.0004      0

### 5. VIF 재확인 

In [25]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# all_cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
#         'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
#         'windchill', 'depart', 'sunrise', 'sunset', 'snowfall' ]
cols = ['cool', 'snowfall', 'preciptotal', 'resultspeed', 'avgspeed']

y = df2_1.loc[:,cols]
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(y.values, i) for i in range(y.shape[1])]
vif["features"] = y.columns
vif = vif.sort_values("VIF Factor", ascending=False).reset_index(drop=True)
vif

,VIF Factor,features
0,25.764382,avgspeed
1,24.651152,resultspeed
2,1.421638,cool
3,1.139324,preciptotal
4,1.001385,snowfall


### 6. VIF 근거로 변수 제외하고  Model 돌림.

In [26]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(cool)\
                            + scale(snowfall) + scale(preciptotal) + C(year) + C(month) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                 1.952e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:39:39   Log-Likelihood:             1.1609e+06
No. Observations:              611815   AIC:                        -2.322e+06
Df Residuals:                  611681   BIC:                        -2.320e+06
Df Model:                         133                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(year)[2012]          0.0004      0

### Anova 분석을 통해 p-value가 0.05보다 큰것은 제외

In [27]:
anova_result2_1 = sm.stats.anova_lm(result2_1).sort_values(by=['PR(>F)'], ascending = False)
anova_result2_1[anova_result2_1['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
scale(preciptotal),1.0,0.005285,0.005285,4.013301,4.514313e-02
C(rainY),1.0,0.135035,0.135035,102.549448,4.225810e-24
C(month),11.0,1.188502,0.108046,82.052681,2.395980e-186
C(year),3.0,193.512423,64.504141,48986.146341,0.000000e+00
C(weekend),1.0,4.540032,4.540032,3447.820151,0.000000e+00
C(store_nbr),5.0,95.217366,19.043473,14462.115868,0.000000e+00
C(item_nbr),110.0,34083.759900,309.852363,235310.057915,0.000000e+00


### 최종 Anova 분석과 TIF 통과한 변수들을 바탕으로 최종 모델생성

In [28]:
# OLS - df2_1_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(preciptotal) + C(rainY) + C(year) + C(month) + \
+ C(item_nbr) + C(weekend) + C(store_nbr) + 0', data = df2_1)

result = model2_1.fit()
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                 1.982e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:40:59   Log-Likelihood:             1.1609e+06
No. Observations:              611815   AIC:                        -2.322e+06
Df Residuals:                  611683   BIC:                        -2.320e+06
Df Model:                         131                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(rainY)[0]            0.0004      0

In [29]:
from patsy import dmatrix
# 독립변수와 종속변수로 나누기
df2_1_X = df2_1.drop(columns=['log1p_units'])
df2_1_target = df2_1['log1p_units']

In [32]:
formula = 'scale(preciptotal) + C(rainY) + C(year) + C(month) + \
+ C(item_nbr) + C(weekend) + C(store_nbr) + 0'
dfX = dmatrix(formula, df2_1_X, return_type='dataframe')
dfy = pd.DataFrame(df2_1_target, columns=["log1p_units"])

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

model = LinearRegression()
cv = KFold(10)

scores = np.zeros(10)
for i, (train_index, test_index) in enumerate(cv.split(dfX)):
    X_train = dfX.values[train_index]
    y_train = dfy.values[train_index]
    X_test = dfX.values[test_index]
    y_test = dfy.values[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores[i] = r2_score(y_test, y_pred)

scores

array([0.97583053, 0.96822077, 0.96797761, 0.97248862, 0.97395525,
       0.96904357, 0.97895072, 0.98241086, 0.98395349, 0.98668642])